In [1]:
from src.mapper import mapProperty, mpRankWTypeSim
from src.wikimedia import searchEntity, searchObjWProperty, searchProperty, searchPropertyRange
import csv
import pandas as pd
from dateutil.parser import parse
import operator
import re
import numpy as np
import sys, os
import scipy.stats 
from dateutil.parser import parse

In [10]:
#GIBRAN THINGS
limit = 5
threshold = 0

def is_date(string):
    try: 
        parse(string)
        return True
    except ValueError:
        return False
    
def ranking(candidateList, goal, flag, propertyLbl):
    scoreList = [0] * len(candidateList)
    for i in range(len(candidateList)):
#         if(not flag):
#             results = searchObjWProperty(candidateList[i]['id'], 'P31')
#             if(len(results['results']['bindings']) > 0):
#                 temp = False
#                 for elem in results['results']['bindings']:
#                     if(elem['itemLabel']['value'].lower() == propertyLbl.lower()):
#                         temp = True
#                         break
#                 if(not temp):
#                     scoreList[i] = scoreList[i] - 5
        results = searchObjWProperty(candidateList[i]['id'], 'P17')
        if(len(results['results']['bindings']) > 0):
            if(results['results']['bindings'][0]['itemLabel']['value'] == 'Indonesia'):
                scoreList[i] = scoreList[i] + 1
        
        lblDis = lDistance(goal, candidateList[i]['label'].lower())
        aliDis = 10000000
        if('aliases' in candidateList[i]):
            aliDis = lDistance(goal, candidateList[i]['aliases'][0].lower())
        
        scoreList[i] = scoreList[i] - min(lblDis, aliDis)
    
    maxScore = max(scoreList)
    if(maxScore < threshold):
        return ''

    idx = scoreList.index(maxScore)
    return candidateList[idx]['id']

def searchID(flag, cell, rowHead):
    json = searchEntity(cell.lower(), limit)['search']
    id = ''
    if(len(json) > 1):
        id = ranking(json, cell.lower(), flag, rowHead)
    elif(len(json) == 1):
        id = json[0]['id']
    
    if(id == '' and flag):
        id = 'QNew'
    elif(id == ''):
        id = 'QNPNew'
        
    return id

def isCordinatLike(col):
    kordinatLike = ['koordinat','kordinat','latitude','longitude','latitude']
    for x in str(col).split(" "):
        print("kata : {} , Kata in KordinatLike {}".format(x,x in kordinatLike))
        if x in kordinatLike:
            return True
    
    return False

    
def makeDatatypeIndex(header_list, df):
    dtMap = []
    for elem in df.loc[0, header_list]:
        try:
            if(elem.replace('.','').replace(',', '').replace('-', '').replace(' ', '').isdigit()):
                dtMap.append(1)
            elif('jl.' in elem.lower()):
                dtMap.append(1)
            elif(is_date(elem)):
                dtMap.append(1)
            else:
                dtMap.append(0)
        except AttributeError:
            dtMap.append(1)
    return dtMap

def makeDatatypeMap(header_list, df):
    reference_row = [str(x) for x in list(df.loc[0, header_list])]
    dtMap = []
    dtColTypes = {}
    index=0
    for elem in reference_row:
        dtColType=""
        pattern_quantity = re.compile("[-+.,()0-9]+")
        pattern_float = re.compile("[0-9\.-]+")
        pattern_globe = re.compile("^[-+]?([1-8]?\d(\.\d+)?|90(\.0+)?),\s*[-+]?(180(\.0+)?|((1[0-7]\d)|([1-9]?\d))(\.\d+)?)$")
        pattern_web = re.compile("[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)")
#         pattern_web = re.compile("^[a-zA-Z0-9_\-\@]+\.[a-zA-Z0-9_\-\.")
        pattern_literal = re.compile("[\.\,\!\?\>\<\/\\\)\(\-\_\+\=\*\&\^\%\$\#\@\!\:\;\~]")
        is_quantity = pattern_quantity.search(elem)
        print("Elem : {} , is_quantity : {}, is_date: {}".format(elem,bool(is_quantity),is_date(elem)))
#         print(list(df.loc[0, header_list]))
        if is_quantity:
            is_float = pattern_float.match(elem)
            is_kordinatLike = isCordinatLike(header_list[reference_row.index(elem)])
            print( is_float)
            if not is_float and is_date(elem):
                 dtColType="Time"
            elif pattern_globe.match(elem):
                 dtColType="GeolocationCoordinate"
            elif is_float and isCordinatLike(header_list[reference_row.index(elem)]):
                dtColType="GeolocationCoordinate"
            else:
                try:
                    x = float(elem)
                    dtColType="Quantity"
                except ValueError:
                    dtColType="String"
        else:
            is_literal_string = bool(pattern_literal.match(elem))
            is_url_string = bool(pattern_literal.match(elem))

            if is_url_string:
                dtColType="URL"
            elif is_literal_string:
                dtColType="String"
            else :
                dtColType="WikibaseItem"

        dtColTypes[header_list[index]]=dtColType 
        index=index+1
    return dtMap, dtColTypes

def lDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

#jalan
def check_protagonist(df):
    ranking = {}
    for col in df.columns:
        entities = set()
        for index, row in df.iterrows():
            entities.add(row[col])
        ranking[col] = len(entities)
    return ranking

def give_verdict_base(df_entity, ranking_diversity):
    ranking = {}
    maxValue = -1
    maxColumn = ""
    for key, value in ranking_diversity.items():
        score = value + len(df_entity.columns)-list(df_entity.columns).index(key)
        if maxValue <= score:
            maxValue = score
            maxColumn = key
        ranking[key] = score
    return maxColumn, maxValue, ranking

def give_verdict_normalized(df_entity, ranking_diversity, weightDiverse=0.5, weightColumnOrder=0.5):
    df_length = df_entity.shape[0]
    ranking = {}
    maxValue = -1
    maxColumn = ""
    for key, value in ranking_diversity.items():
        diverseScore = value/df_length
        columnOrderScore = 1 - (list(df_entity.columns).index(key)/len(df_entity.columns))
        score = (weightDiverse * diverseScore) + (weightColumnOrder * columnOrderScore)
        if maxValue <= score:
            maxValue = score
            maxColumn = key
        ranking[key] = score
    return maxColumn, maxValue, ranking

#tie breaker on highest value . but the highest value has multiplice instances
def give_verdict_columntb(df_entity, ranking_diversity):
    df_length = df_entity.shape[0]
    ranking = {}
    maxValue = -1 #value can not be lower or equal to zero , so this is safe initial poin
    maxColumns = []
    for key, value in ranking_diversity.items():
        if value > maxValue:
            maxColumns=[key]
            maxValue = value
        elif value == maxValue:
            maxColumns.append(key)
        else:
            pass
        ranking[key] = value

    if len(maxColumns) > 1:
        maxOrderScore = -1  #value can not be lower or equal to zero , so this is safe initial poin
        for col in maxColumns:
            columnOrderScore = 1 - (list(df_entity.columns).index(key)/len(df_entity.columns))
            if columnOrderScore > maxOrderScore:
                maxColumns[0] = col
                maxOrderScore = columnOrderScore
            ranking[col] = ranking[col]+columnOrderScore
    return maxColumns[0], maxValue, ranking

#tie breaker on , if any, highest value that have multiplie instances ( So it could be , not the highest value , but highest value that has multiple instances)

#tie breaker on any multiple instances


#entropy
def give_verdict_entropy(df_entity):
    maxEntropy = -1
    maxColumn=""
    ranking={}
    for col in df_entity.columns:
        dummy, counts = np.unique(df_entity[col].astype(str), return_counts=True)
        probs = []
        numbers_of_data=df_entity.shape[0]
        for occ in counts:
            probs.append(occ/numbers_of_data)
        entropy = scipy.stats.entropy(probs)#base = e
#         print(str(type(entropy))+ " " + str(maxEntropy))
        if entropy > maxEntropy:
            maxEntropy=entropy
            maxColumn=col
        ranking[col]=entropy
    return maxColumn, maxEntropy, ranking

# Jalan MAIN
def determine_protagonist(df, dtMap):
    entity_columns = dtMap
    df_entity = df[entity_columns]
    ranking = check_protagonist(df_entity)
    hasil={}
    hasil['base'], score, info = give_verdict_base(df_entity, ranking)
    hasil['normalize-0.5:0.5'], score, info = give_verdict_normalized(df_entity, ranking)
    hasil['normalize-0.7:0.3'], score, info = give_verdict_normalized(df_entity, ranking, 0.7, 0.3)
    hasil['normalize-0.3:0.7'], score, info = give_verdict_normalized(df_entity, ranking, 0.3, 0.7)
    hasil['base-columntb'], score, info = give_verdict_columntb(df_entity, ranking)
    hasil['entropy'], score, info = give_verdict_entropy(df_entity)



    return hasil
#     for keys, value in hasil.items():
#         print(str(keys) + " : " + str(value))

def check_wb_type(property_id):
    if len(property_id) <= 1 or property_id[0] != 'P':
        return ''
    res = searchPropertyRange(property_id)
    wd_type = res['results']['bindings'][0]['wbtype']['value']
    return wd_type[wd_type.find('#')+1:]

In [15]:
os.listdir("data/uncleaned")

['SMKRawanBanjir.csv',
 'daftar-taman-kota-bandung.csv',
 'Data-Pemakaman-di-Provinsi-DKI-Jakarta.csv',
 'daftar-tk-di-kecamatan-buah-batu.csv',
 'Data-Sekolah-Menengah-Atas-Provinsi-DKI-Jakarta.csv',
 'Data-Waduk-2018.csv',
 'ruang-laktasi.csv']

In [42]:
nama_folder = "data/uncleaned/"
nama_csv = "daftar-taman-kota-bandung.csv"
nama_file = nama_folder + nama_csv
df = pd.read_csv(nama_file, encoding='latin-1')
df_asli_column_asli = df.columns
df.columns
df.iloc[:,0:]

,nama_tempat,lokasi,kategori,kelurahan,kecamatan,sub_wilayah_kota,tipe,latitude,longitude
0,Pet Park,Jl. Cisangkuy,Taman,Cihapit,Bandung Wetan,Cibeunying,Taman Tematik,-6.904864,107.626810
1,Skate Park,Jl. Tamansari,Taman,Lebak Siliwangi,Coblong,Cibeunying,Taman Tematik,-6.898417,107.607814
2,Taman Alun Alun Ujung Berung,Lapangan Ujung Berung,Taman,Cigending,Ujungberung,Ujungberung,Taman Tematik,-6.913927,107.701269
3,Taman Balai Kota,Jl. Wastukencana,Taman,Babakan Ciamis,Sumur Bandung,Cibeunying,Taman Tematik,-6.913041,107.609618
4,Taman Braga,Jl. Naripan,Taman,Braga,Sumur Bandung,Cibeunying,Taman Tematik,-6.919793,107.610089
5,Taman Cibeunying,Jl. Cibeunying,Taman,Cihapit,Bandung Wetan,Cibeunying,Taman Tematik,-6.903945,107.623269
6,Taman Dago,Jl. Ir. H. Djuanda,Taman,Lebak Siliwangi,Coblong,Cibeunying,Taman Tematik,-6.898593,107.612464
7,Taman Film,Jl. Tamansari,Taman,Lebak Siliwangi,Coblong,Cibeunying,Taman Tematik,-6.898417,107.607814
8,Taman Fitness,Jl. Teuku Umar,Taman,Lebak Gede,Coblong,Cibeunying,Taman Tematik,-6.891965,107.615722
9,Taman Fotografi,Jl. Anggrek,Taman,Merdeka,Sumur Bandung,Cibeunying,Taman Tematik,-6.913519,107.627205


In [43]:
df=df.iloc[:,0:2]
df.columns=['a','a']
temp = df
col1 = temp.iloc[:,0]
col2 = temp.iloc[:,1]
temp_col = "@"+col1 + "/"+col2
df['baru']=temp_col
df.drop(columns=['a'], inplace=True)
df.shape[1]

1

In [8]:
header_list = [x.replace("_"," ").lower() for x in list(df.columns)]
df.columns=header_list
if 'no.' in header_list:
    header_list.remove('no.')
# header_list=[x.lower() for x in header_list]
header_list

['tahun', 'nama sekolah ', 'kota', 'predikat']

In [11]:
dtMap, dt_type = makeDatatypeMap(header_list, df)
dt_type

Elem : 2018.0 , is_quantity : True, is_date: False
kata : tahun , Kata in KordinatLike False
<_sre.SRE_Match object; span=(0, 6), match='2018.0'>
kata : tahun , Kata in KordinatLike False
Elem : tk. tzu chi , is_quantity : True, is_date: False
kata : nama , Kata in KordinatLike False
kata : sekolah , Kata in KordinatLike False
kata :  , Kata in KordinatLike False
None
Elem : jakarta utara , is_quantity : False, is_date: False
Elem : juara 1 , is_quantity : True, is_date: False
kata : predikat , Kata in KordinatLike False
None


{'kota': 'WikibaseItem',
 'nama sekolah ': 'String',
 'predikat': 'String',
 'tahun': 'Quantity'}

In [16]:
hasil_verdict = determine_protagonist(df, dtMap)
protagonist = hasil_verdict['entropy']
protagonist

'nama sekolah'

In [17]:
dt_type.pop(protagonist)
header_list.remove(protagonist)
types_list = []
for head in header_list:
    types_list.append(dt_type[head])
mapping, mappingLabel = mpRankWTypeSim(header_list, types_list)
mapping[protagonist]=protagonist
header_list.append(protagonist)
mapping

kicut
Connected to ES Service


{'alamat': 'P6375',
 'jumlah guru': 'P1128',
 'jumlah siswa': 'P2196',
 'kecamatan': 'P131',
 'kelurahan': 'P131',
 'kondisi lingkungan': 'P1196',
 'lokasi geografis': '',
 'nama sekolah': 'nama sekolah',
 'telp sekolah': ''}

In [18]:
header_list

['alamat',
 'kelurahan',
 'kecamatan',
 'jumlah siswa',
 'jumlah guru',
 'telp sekolah',
 'kondisi lingkungan',
 'lokasi geografis',
 'nama sekolah']

In [19]:
# header_list.remove(protagonist)
# mapping,xa = mapProperty(header_list, '')
# mapping[protagonist] = protagonist
# header_list.append(protagonist)
# xa

In [20]:
dtMap = makeDatatypeIndex(header_list, df)
dtMap
# header_list

[1, 0, 0, 1, 1, 1, 0, 1, 0]

In [21]:
convertMap = {} 
finalMap = {} 
print("Start processing Table") 
counter = 2 
for header in header_list: 
    if(header in mapping): 
        print("Processing {} Column".format(header)) 
        columnList = [] 
        for cell in df[header]: 
            if(dtMap[header_list.index(header)]): 
                id = cell 
            else: 
                temp = "{}{}".format(header, cell) 
                if(temp in convertMap): 
                    id = convertMap[temp] 
                else: 
                    id = searchID(protagonist == header, str(cell), header) 
                    convertMap[temp] = id 
            columnList.append(id) 
        print("finished processing {} datas".format(len(columnList))) 
        if(header == protagonist): 
            finalMap[header] = columnList 
        else: 
            if(mapping[header] in finalMap): 
                finalMap['{}({})'.format(mapping[header], counter)] = columnList 
                counter = counter + 1 
            else: 
                finalMap[mapping[header]] = columnList 

print("All task Finished")

Start processing Table
Processing alamat Column
finished processing 25 datas
Processing kelurahan Column
finished processing 25 datas
Processing kecamatan Column
finished processing 25 datas
Processing jumlah siswa Column
finished processing 25 datas
Processing jumlah guru Column
finished processing 25 datas
Processing telp sekolah Column
finished processing 25 datas
Processing kondisi lingkungan Column
finished processing 25 datas
Processing lokasi geografis Column
finished processing 25 datas
Processing nama sekolah Column
finished processing 25 datas
All task Finished


In [23]:
new_df = pd.DataFrame.from_dict(finalMap)
new_df
# convertMap

,P6375,P131,P131(2),P2196,P1128,,P1196,(3),nama sekolah
0,Jl. Bendera Raya - Intisari 3,Q12488768,Q193374,350,39.0,021-8720108,Q8068,"-6.331491, 106.858711",QNew
1,Jl. Karya Barat I Ujung No.24 Jakarta Barat,Q7999779,Q192723,30,13.0,82114486336,Q8068,"-6.161085, 106.773727",QNew
2,"Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng Barat",Q5058847,Q192714,60,10.0,(021)5447107,Q8068,"-6.141474, 106.725876",QNew
3,Jl. Kebon Nanas Utara II/23,Q12479264,Q193323,295,21.0,021-85914453,Q8068,"1.000000, 2.000000",Q31778996
4,Jl. Tabah Raya Komp. TNI AL,Q2806467,Q193545,512,28.0,(021)4535140,Q8068,"-6.158968, 106.885063",QNew
5,Jl. Let. Jend. S. Parman No. 69,Q13097193,Q192745,0,14.0,(021)70624646,Q8068,"-6.193888, 106.797432",QNew
6,Jl. Keutamaan No. 89,Q12492672,Q192748,144,14.0,(021) 6344508,Q8068,"-6.156000, 106.814003",QNew
7,Jl. Mindi No. 36,Q6472418,QNPNew,55,13.0,43909933,Q8068,"-6.113743, 106.916031",Q60610068
8,Jl. Kesatuan No.1 kp.Klingkit,Q4201202,Q192714,306,18.0,(021)58300174,Q8068,"-6.156400, 106.728104",Q20427513
9,Jl. Bendera Raya / Intisari III No.17,Q12488768,Q193374,209,22.0,(021)8728766,Q8068,"-6.330335, 106.858643",Q12510798


In [24]:
new_df.to_csv('data/dummymapping.csv')

In [33]:
df_asli

,No.,Nama Sekolah,Alamat,Kelurahan,Kecamatan,Jumlah Siswa,Jumlah Guru,Telp Sekolah,Kondisi Lingkungan,Lokasi Geografis
0,1,SMK Analis Kesehatan Tunas Harapan,Jl. Bendera Raya - Intisari 3,Kalisari,Pasar Rebo,350,39.0,021-8720108,Banjir,"-6.331491, 106.858711"
1,2,SMK Bhakti,Jl. Karya Barat I Ujung No.24 Jakarta Barat,Wijaya Kusuma,Grogol Petamburan,30,13.0,82114486336,Banjir,"-6.161085, 106.773727"
2,3,SMK Bunga Nusa I,"Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng Barat",Cengkareng Barat,Cengkareng,60,10.0,(021)5447107,Banjir,"-6.141474, 106.725876"
3,4,SMK Dewi Sartika,Jl. Kebon Nanas Utara II/23,Cipinang Cempedak,Jatinegara,295,21.0,021-85914453,Banjir,"1.000000, 2.000000"
4,5,SMK Hang Tuah I Jakarta,Jl. Tabah Raya Komp. TNI AL,Kelapa Gading Barat,Kelapa Gading,512,28.0,(021)4535140,Banjir,"-6.158968, 106.885063"
5,6,SMK Ibu Pertiwi 1 Jakarta,Jl. Let. Jend. S. Parman No. 69,Slipi,Palmerah,0,14.0,(021)70624646,Banjir,"-6.193888, 106.797432"
6,7,SMK Islam Fatahillah Jakarta,Jl. Keutamaan No. 89,Krukut,Taman Sari,144,14.0,(021) 6344508,Banjir,"-6.156000, 106.814003"
7,8,SMK Karya Bahari,Jl. Mindi No. 36,Lagoa,K o j a,55,13.0,43909933,Banjir,"-6.113743, 106.916031"
8,9,SMK Kesatuan,Jl. Kesatuan No.1 kp.Klingkit,Rawa Buaya,Cengkareng,306,18.0,(021)58300174,Banjir,"-6.156400, 106.728104"
9,10,SMK Kimia Tunas Harapan,Jl. Bendera Raya / Intisari III No.17,Kalisari,Pasar Rebo,209,22.0,(021)8728766,Banjir,"-6.330335, 106.858643"


In [36]:
df_asli = df
df = pd.read_csv("data/dummymapping.csv")
protagonist = protagonist
df.head()

,Unnamed: 0,P6375,P131,P131(2),P2196,P1128,Unnamed: 6,P1196,(3),nama sekolah
0,0,Jl. Bendera Raya - Intisari 3,Q12488768,Q193374,350,39.0,021-8720108,Q8068,"-6.331491, 106.858711",QNew
1,1,Jl. Karya Barat I Ujung No.24 Jakarta Barat,Q7999779,Q192723,30,13.0,82114486336,Q8068,"-6.161085, 106.773727",QNew
2,2,"Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng Barat",Q5058847,Q192714,60,10.0,(021)5447107,Q8068,"-6.141474, 106.725876",QNew
3,3,Jl. Kebon Nanas Utara II/23,Q12479264,Q193323,295,21.0,021-85914453,Q8068,"1.000000, 2.000000",Q31778996
4,4,Jl. Tabah Raya Komp. TNI AL,Q2806467,Q193545,512,28.0,(021)4535140,Q8068,"-6.158968, 106.885063",QNew


In [37]:
def identify_literal_columns(df):
    literal_columns = []
    samplerow = df.shape[0]
    a = df.loc[0:samplerow,:]
    for col in a.columns:
        isLiteral = False
        for index, row in a.iterrows():
#             print("==========> " + row[[col]])
            pattern = re.compile("^Q([1-9]+)")#match the Q123123 pattern of entity name
            
            if pattern.match(str(row[col])):
                continue
            elif str(row[col]) == "QNPNew" or str(row[col]) == "QNew":
                continue
            else:
                isLiteral = True
        if isLiteral and col != protagonist:
#             print(col)
            literal_columns.append(col)
    return literal_columns
    
def identify_double_columns(df):
    double_columns = {}
    columns = df.columns
    for col in columns:
        if "(" in col:
            print(col + " " + col[:col.find("(")])
            double_columns[col] = col[:col.find("(")]
    return double_columns

In [53]:
df_qs = pd.DataFrame(df)
df_qs=df_qs.loc[:, ~df_qs.columns.str.contains('^Unnamed')] #drop unnamed col (index)
df_qs

,P6375,P131,P131(2),P2196,P1128,P1196,(3),nama sekolah
0,Jl. Bendera Raya - Intisari 3,Q12488768,Q193374,350,39.0,Q8068,"-6.331491, 106.858711",QNew
1,Jl. Karya Barat I Ujung No.24 Jakarta Barat,Q7999779,Q192723,30,13.0,Q8068,"-6.161085, 106.773727",QNew
2,"Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng Barat",Q5058847,Q192714,60,10.0,Q8068,"-6.141474, 106.725876",QNew
3,Jl. Kebon Nanas Utara II/23,Q12479264,Q193323,295,21.0,Q8068,"1.000000, 2.000000",Q31778996
4,Jl. Tabah Raya Komp. TNI AL,Q2806467,Q193545,512,28.0,Q8068,"-6.158968, 106.885063",QNew
5,Jl. Let. Jend. S. Parman No. 69,Q13097193,Q192745,0,14.0,Q8068,"-6.193888, 106.797432",QNew
6,Jl. Keutamaan No. 89,Q12492672,Q192748,144,14.0,Q8068,"-6.156000, 106.814003",QNew
7,Jl. Mindi No. 36,Q6472418,QNPNew,55,13.0,Q8068,"-6.113743, 106.916031",Q60610068
8,Jl. Kesatuan No.1 kp.Klingkit,Q4201202,Q192714,306,18.0,Q8068,"-6.156400, 106.728104",Q20427513
9,Jl. Bendera Raya / Intisari III No.17,Q12488768,Q193374,209,22.0,Q8068,"-6.330335, 106.858643",Q12510798


In [54]:
# [CAREFULL]
# # CELL TO ADD MANUALLY ADDED PROPERTY

# df_qs.columns = [x if x!='(3)' else 'P625' for x in df_qs.columns]

In [55]:

literal_columns = identify_literal_columns(df_qs)
double_columns = identify_double_columns(df_qs)
df_qs.rename({protagonist:'qid'}, axis=1, inplace=True)

# print(df_qs.columns[0])
#ngereplace QID(protagonist) yg sifat Qnew
df_qs.replace(["QNew","QNPNew"],"",inplace=True)

df_qs.columns=[c if c not in double_columns.keys() else double_columns[c] for c in df_qs.columns]

print(literal_columns)
#nambain label dari csv asli sama nambain quote untuk literal columns
df_qs['Lid']=df_asli[protagonist]



print("START checking property range")
for liter_col in literal_columns:
    print("checking for {}".format(liter_col))
    range_type = check_wb_type(liter_col)
#      if dt_type[mapping[liter_col]] == 'String':
    #  buat mbedain keluarga besar string
    if range_type == 'String':
        df_qs[liter_col]="\"\"\"\"" + df_qs[liter_col] + "\""
    elif range_type == 'Monolingualtext':
        df_qs[liter_col]="id:\"" + df_qs[liter_col] + "\""
    elif range_type == 'GlobeCoordinate':
        temp = list(df_qs[liter_col])
        temp = ["@"+x.replace(",","/").replace(" ","") for x in temp]
        df_qs[liter_col]=temp
        
print("END checking property range")
  
#     pattern = re.compile("^-?[0-9][0-9,\.]+$")
#     pattern = re.compile(".*[a-zA-Z].*")

#pemindahan QID ke depan
df_qs=df_qs[['qid'] + [c for c in df_qs if c != 'qid']]

P131(2) P131
['P6375', 'P2196', 'P1128', 'P625']
START checking property range
checking for P6375
checking for P2196
checking for P1128
checking for P625
END checking property range


In [56]:
df_qs.head()


,qid,P6375,P131,P131,P131,P131,P2196,P1128,P1196,P625,Lid
0,,"id:""Jl. Bendera Raya - Intisari 3""",Q12488768,Q193374,Q12488768,Q193374,350,39.0,Q8068,@-6.331491/106.858711,SMK Analis Kesehatan Tunas Harapan
1,,"id:""Jl. Karya Barat I Ujung No.24 Jakarta Barat""",Q7999779,Q192723,Q7999779,Q192723,30,13.0,Q8068,@-6.161085/106.773727,SMK Bhakti
2,,"id:""Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng B...",Q5058847,Q192714,Q5058847,Q192714,60,10.0,Q8068,@-6.141474/106.725876,SMK Bunga Nusa I
3,Q31778996,"id:""Jl. Kebon Nanas Utara II/23""",Q12479264,Q193323,Q12479264,Q193323,295,21.0,Q8068,@1.000000/2.000000,SMK Dewi Sartika
4,,"id:""Jl. Tabah Raya Komp. TNI AL""",Q2806467,Q193545,Q2806467,Q193545,512,28.0,Q8068,@-6.158968/106.885063,SMK Hang Tuah I Jakarta


In [57]:
# # [CAREFULL]
# # CELL TO ADD MANUALLY ADDED PROPERTY
# df_qs.columns = list(df_qs.columns).replace('',)
df_qs.head()


,qid,P6375,P131,P131,P131,P131,P2196,P1128,P1196,P625,Lid
0,,"id:""Jl. Bendera Raya - Intisari 3""",Q12488768,Q193374,Q12488768,Q193374,350,39.0,Q8068,@-6.331491/106.858711,SMK Analis Kesehatan Tunas Harapan
1,,"id:""Jl. Karya Barat I Ujung No.24 Jakarta Barat""",Q7999779,Q192723,Q7999779,Q192723,30,13.0,Q8068,@-6.161085/106.773727,SMK Bhakti
2,,"id:""Jl. Rawa Bengkel Sdn 19 Pagi, Cengkareng B...",Q5058847,Q192714,Q5058847,Q192714,60,10.0,Q8068,@-6.141474/106.725876,SMK Bunga Nusa I
3,Q31778996,"id:""Jl. Kebon Nanas Utara II/23""",Q12479264,Q193323,Q12479264,Q193323,295,21.0,Q8068,@1.000000/2.000000,SMK Dewi Sartika
4,,"id:""Jl. Tabah Raya Komp. TNI AL""",Q2806467,Q193545,Q2806467,Q193545,512,28.0,Q8068,@-6.158968/106.885063,SMK Hang Tuah I Jakarta


In [58]:
df_qs.to_csv('data/result/qs_file_result-{}.csv'.format(nama_csv), index=False)

In [ ]:
import os 
os.remove("data/quickstatements_file.csv") 
print("File Removed!")